In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import geopandas as gpd
from datetime import date
from datetime import datetime
import pyodbc
import pandas.io.sql as psql

#Set viewing options in Jupyter
np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_columns', 999)
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
#Pull the SQL database into Python and use the query they have already built for you from the download page.

con = pyodbc.connect(driver='{SQL Server Native Client 11.0}',  #<-Make sure your driver matches whatever you put in here
                      server='LAPTOP-XXXXXXXX\SQLEXPRESS', #<-Find this at the opening screen of MS SQL Server Mgmt. Studio
                      database='FracFocusRegistry', #<-If you made no changes on import of the FF db backup, this is it.
                      trusted_connection='yes') 
cursor = con.cursor()

sql = """SELECT * From FracFocusRegistry.dbo.RegistryUpload ru Join FracFocusRegistry.[dbo].[RegistryUploadIngredients] ri

on ru.pkey=ri.pKeyDisclosure 

Left JOIN FracFocusRegistry.dbo.RegistryUploadPurpose rp

on ri.pKeyPurpose = rp.pKey

Where (ri.pKeyPurpose is null or ru.pkey = ri.pKeyDisclosure);"""

#    you can specify states and counties by adding something like this example:  

#    StateNumber = (30) AND (CountyNumber = 25 OR CountyNumber = 15)

#    in this spot of the query: Where (ri.pKeyPurpose is null or ru.pkey = ri.pKeyDisclosure) --put code here-- ;""" 

FFNMdata = pd.read_sql(sql, con)
con.close()
FFNMdata.info()

In [ ]:
#With the way the SQL query FracFocus has set up, you can run into an issue with duplicate column names. Fix it with this.

def df_columnrename(df):
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 0
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df

#Apply the column rename function to the dataframe
FFNMdata = df_columnrename(FFNMdata)

In [ ]:
#There are \n(newline) and \t(tab) tags on some entries. For uniformity, capitalize the ingredients (or everything), 
#get rid of nan's, and get rid of those tags with this cell.

FFNMdata['IngredientName'] = FFNMdata['IngredientName'].str.upper()
FFNMdata['IngredientName'] = FFNMdata['IngredientName'].replace('\n','', regex=True)
FFNMdata['IngredientName'] = FFNMdata['IngredientName'].replace('\t','', regex=True)

FFNMdata['PercentHFJob'] = FFNMdata['PercentHFJob'].fillna(0.0)


In [ ]:
#Number of wells in database

APIvals = np.unique(FFNMdata['APINumber'])
print("Database Wellcount:", len(APIvals))

In [ ]:
#PercentHFJob will give you frac compositions, and there are many entry errors throughout the database in this column.
#We will check for sums equaling, or close to, 1.0, 100.0, and an arbitrary amount I thought may be present below.

Sumcheck = FFNMdata.groupby('APINumber')['PercentHFJob'].sum()
Sumcheck = pd.DataFrame(Sumcheck)
Sumcheck.reset_index(inplace=True)
Sumcheck.head(5)

In [ ]:
print("Number of wells with a 1% tolerance")
DecimalSumdf = Sumcheck[(Sumcheck['PercentHFJob']>=.99) & (Sumcheck['PercentHFJob']<1.01)]
WholeSumdf = Sumcheck[(Sumcheck['PercentHFJob']>=99.0) & (Sumcheck['PercentHFJob']<=101.0)]
TooMuchSumdf = Sumcheck[(Sumcheck['PercentHFJob']>=110)]
print("Number of decimal# compositions:", len(DecimalSumdf))
print("Number of whole# compositions:", len(WholeSumdf))
print("Number of way too much compositions:", len(TooMuchSumdf))
print("Total count:", len(DecimalSumdf) + len(WholeSumdf) + len(TooMuchSumdf))
print("Total good for use:", len(DecimalSumdf) + len(WholeSumdf))

In [ ]:
print("Number of wells with a 5% tolerance")
DecimalSumdf = Sumcheck[(Sumcheck['PercentHFJob']>=.90) & (Sumcheck['PercentHFJob']<1.1)]
WholeSumdf = Sumcheck[(Sumcheck['PercentHFJob']>=90.0) & (Sumcheck['PercentHFJob']<=110.0)]
TooMuchSumdf = Sumcheck[(Sumcheck['PercentHFJob']>110.0)]
print("Number of decimal# compositions:", len(DecimalSumdf))
print("Number of whole# compositions:", len(WholeSumdf))
print("Number of way too much compositions:", len(TooMuchSumdf))
print("Total count:", len(DecimalSumdf) + len(WholeSumdf) + len(TooMuchSumdf))
print("Total good for use:", len(DecimalSumdf) + len(WholeSumdf))

In [ ]:
print("Number of wells that are exactly 1, 100, or 110 (the 110 is just a guess)")
DecimalSumdf = Sumcheck[(Sumcheck['PercentHFJob']==1.0)]
WholeSumdf = Sumcheck[(Sumcheck['PercentHFJob']== 100.0)]
TooMuchSumdf = Sumcheck[(Sumcheck['PercentHFJob']== 110.0)]
print("Number of decimal# compositions:", len(DecimalSumdf))
print("Number of whole# compositions:", len(WholeSumdf))
print("Number of way too much compositions:", len(TooMuchSumdf))
print("Total count:", len(DecimalSumdf) + len(WholeSumdf) + len(TooMuchSumdf))
print("Total good for use:", len(DecimalSumdf) + len(WholeSumdf))

In [ ]:
#In the interest of sticking to realistic numbers, let's use the 99 - 101 range in terms of a sum cut off.
#Granted, there are notes telling you that sums may add to more than 100%, I would rather just stick
#with a tighter tolerance for this post.

#You can increase/decrease your usable well count by doing more cleaning and inspection yourself.

DecimalSumdf = Sumcheck[(Sumcheck['PercentHFJob']>=.99) & (Sumcheck['PercentHFJob']<1.01)]
WholeSumdf = Sumcheck[(Sumcheck['PercentHFJob']>=99.0) & (Sumcheck['PercentHFJob']<=101.0)]

In [ ]:
#Convert decimal percent wells to whole number percentages before combining the filtered data sets
DecimalConvert = FFNMdata[FFNMdata["APINumber"].isin(DecimalSumdf['APINumber'])]

DecimalConvert['PercentHFJob_fix'] = [(DecimalConvert['PercentHFJob'].iloc[x])*100.0 if 
                                      (DecimalConvert['PercentHFJob'].iloc[x] <= 1.0) 
                                      else DecimalConvert['PercentHFJob'].iloc[x] for x in range(0,len(DecimalConvert))]

DecimalConvert = DecimalConvert.drop(['PercentHFJob', 'PercentHFJob_1'], axis=1)
DecimalConvert = DecimalConvert.rename(columns={"PercentHFJob_fix":'PercentHFJob'})

In [ ]:
WholeValuedf = FFNMdata[FFNMdata["APINumber"].isin(WholeSumdf['APINumber'])]
WholeValuedf = WholeValuedf.drop(['PercentHFJob_1'], axis=1)

In [ ]:
#Put the converted decimal to whole number and original whole number dataframes back together

TotalUsedf = pd.concat([DecimalConvert, WholeValuedf], axis = 0)
TotalUsedf = TotalUsedf.sort_values(by = "APINumber")
APIvals = np.unique(TotalUsedf['APINumber'])
print("Well count for db filter:", len(APIvals), "wells")

In [ ]:
#Create a dataframe for water volumes that has only the columns you need

FFwatervol = TotalUsedf[["APINumber", "WellName", "TotalBaseWaterVolume", "OperatorName", "StateName", "StateNumber", 
                         "CountyName", "CountyNumber", "JobEndDate", "Latitude", "Longitude", "Projection"]]
FFwatervol = FFwatervol.sort_values(by = ["APINumber", "TotalBaseWaterVolume"], ascending=True)
FFwatervol = FFwatervol.drop_duplicates(subset='APINumber', keep='last', inplace=False)
FFwatervol = FFwatervol.reset_index(drop=True)

In [ ]:
#Create a dataframe for propant amounts to then apply the filter list below

FFPropvol = TotalUsedf[["APINumber", "IngredientName", "PercentHFJob"]]

In [ ]:
#Polylactic acid is used as a diverter, but it looked a little suspect when going through the database, so I included it
#If it doesn't matter, the numbers will be inconsequential in considering the mass of the proppant.

#You can also just do a text search to break these out, but I already do various lists for looking at other aspects of
#what is in the frac mixtures, so a cut and paste is much easier - for me.

Proppantlist = ["  CRYSTALLINE SILICA, QUARTZ    ", 
"  CRYSTALLINE SILICA: QUARTZ (SIO2)",
" CRYSTALLINE SILICA  QUARTZ ", 
" CRYSTALLINE SILICA (QUARTS) ", 
" CRYSTALLINE SILICA QUARTZ", 
" CRYSTALLINE SILICA: QUARTZ (SIO2)", 
" CRYSTALLINE SILICA: QUARTZ (SIO2) CRYSTALLINE SILICA: QUARTZ (SIO2)",
" INTERMEDIATE & HIGH STRENGTH PROPPANTS (ALL SIZES) ", 
" ORGANICALLY TREATED CLAY", 
" SILICA, CRYSTALLINE - QUARTZ ", 
" SILICA, CRYSTALLINE QUARTZ", 
" ULEXITE", 
"CRYSTALLINE SILICA (QUARTZ)", '"CRYSTALLINE SILICA QUARTZ"',
'"CRYSTALLINE SILICA: QUARTZ (SIO2) CRYSTALLINE SILICA: QUARTZ (SIO2)"', 
'"CRYSTALLINE SILICA: QUARTZ (SIO2)"', 
'"CRYSTALLINE SILLICA: QUARTZCRYSTALLINE SILLICA: QUARTZ"', 
'"SILICA, CRYSTALLINE- QUARTS QUARTZ QUARTZ"', 
'"ULEXITE"', 
'"ULEXITEULEXITEULEXITE"',
"10 MESH STANDARD POWDER", 
"12/20 SAND", 
"12-20 MESH BRADY SAND_OKLAHOMA", 
"12-20 MESH OTTAWA SD., TX", 
"12-40 MESH OTTAWA SD.,TX", 
"14/40 FRACBLACK HT", 
"16/20 EL PROP", 
"16/30 SAND", 
"16-30 MESH BRADY SAND_OKLAHOMA", 
"20/40 ECONOPROP", 
"20/40 MG LITE", 
"20/40 OTTAWA SAND", 
"20/40 PC", 
"20/40 PRC", 
"20/40 SINTERLITE", 
"20/40 SUPER LC", 
"20/40 TG", 
"20/40 WHITE", 
"20-40 MESH OTTAWA SAND_TX", 
"20-40 MESH OTTAWA SD TX", 
"30/50 PPC", 
"30/50 SAND", 
"30/50 SUPER LC", 
"30/50 TEMPERED HS", 
"30/50 TG", 
"30/50 WHITE", 
"33-40", 
"35# SIROCCO", 
"40/70 NORTHERN WHITE SAND", 
"40/70 PPC", 
"40/70 PREFERRED SAND", 
"40/70 TEMPERED HS", 
"40/70 WHITE", 
"40/70 WHITE SAND", 
"40/80 HYDROPROP", 
"AA-400 (ALUMINUM OXIDE)", 
"ALUMINATE (AL(OH)63-) (OC-6-11)- MAGNESIUM CARBONATE HYDROXIDE (2:6:1:4)", 
"ALUMINION OXIDE", 
"ALUMINIUM OXIDE", 
"ALUMINIUM SILICATE", 
"ALUMINUIUM OXIDE", 
"ALUMINUM NEEDLES", 
"ALUMINUM OXIDE", 
"ALUMINUM OXIDE ", 
"ALUMINUM OXIDE / CORUNDUM", 
"ALUMINUM OXIDE CORUNDUM", 
"ALUMINUM OXIDE CORUNDUM 35-65", 
"ALUMINUM OXIDE, CORUNDUM", 
"ALUMINUM OXIDE/CORUNDUM", 
"ALUMINUM OXIDE/CURUNDUM", 
"ALUMINUM SILCATE/MULITE", 
"ALUMINUM SILICATE", 
"ALUMINUM SILICATE (MULLITE)", 
"ALUMINUM SILICATE / MULLITE", 
"ALUMINUM SILICATE MULITE", 
"ALUMINUM SILICATE MULLITE", 
"ALUMINUM SILICATE MULLITE 35-65", 
"ALUMINUM SILICATE, MULITE", 
"ALUMINUM SILICATE/ MULLITE", 
"ALUMINUM SILICATE/MULITE", 
"ALUMINUM SILICATE/MULLITE", 
"ALUMIUM OXIDE", 
"AMORPHOUS SILICA", 
"AMORPHOUS SILICA (GLASS)", 
"AMORPHOUS SILICA (LIQUIFIED)", 
"AMORPHOUS SILICA/ETHYLENE GLYCOL", 
"AMORPHOUS SILISCA", 
"AMORPHUS SILICA", 
"BAUXITE", 
"BAUXITE (AL2O3.XH2O), CALCINED", 
"BAUXITE (AL2O3.XH2O),CALCINED", 
"BAUXITE CALCINED", 
"BAUXITE, CALCINED", 
"CALCINED BAUXITE", 
"CALCINED KAOLIN (MULLITE)",
"CARBAMIDE", 
"CERAMIC", 
"CERAMIC BEADS", 
"CERAMIC MATERIAL & WARES CHEMICALS ", 
"CERAMIC MATERIAL & WARES, CHEMICALS", 
"CERAMIC MATERIAL & WARES, CHEMICALS ", 
"CERAMIC MATERIAL AND WARES CHEMCIAL", 
"CERAMIC MATERIAL AND WARES CHEMICAL", 
"CERAMIC MATERIAL AND WARES, CHEMCIAL", 
"CERAMIC MATERIAL AND WARES, CHEMICAL", 
"CERAMIC MATERIALS", 
"CERAMIC MATERIALS AND WARES", 
"CERAMIC MATERIALS AND WARES CHEMCICALS", 
"CERAMIC MATERIALS AND WARES CHEMICAL", 
"CERAMIC MATERIALS AND WARES CHEMICALS", 
"CERAMIC MATERIALS AND WARES CHEMICALS", 
"CERAMIC MATERIALS AND WARES, CHEMCICALS", 
"CERAMIC MATERIALS AND WARES, CHEMICAL", 
"CERAMIC MATERIALS AND WARES, CHEMICALS", 
"CERAMIC MATERIAS AND WARES CHEMICALS", 
"CERAMIC MATERIAS AND WARES, CHEMICALS", 
"CERAMIC MATERICLAS AND WARES CHEMICALS", 
"CERAMIC MATERICLAS AND WARES, CHEMICALS", 
"CERAMIC MATIERIALS AND WARES ", 
"CERAMIC MICROSPHERE", 
"CERAMIC MICROSPHERES", 
"CERAMIC MICROSPHERES/GLUTARALDEHYDE", 
"CERAMIC MIX", 
"CERAMIC PROPANT", 
"CERAMIC PROPPANT", 
"CERAMIC PROPPANT PROPRIETARY", 
"CERAMICM MATERIALS AND WARES CHEMICALS", 
"CERAMICM MATERIALS AND WARES, CHEMICALS", 
"CERAMICS MATERIALS AND WARES, CHEMICALS", 
"CHRISTOBALITE", 
"CHROMITE SAND", 
"CHRYSTALLINE SILICA", 
"CHRYSTALLINE SILICA, QUARTZ", 
"CHRYSTALLINE SILICA: CRISTOBALITE", 
"CHRYSTALLINE SILICA: CRISTOBALITE ", 
"CHRYSTALLINE SILICA: QUARTZ", 
"CHRYSTALLINE SILICA:CRISTOBALITE ", 
"CORRUNDUM", 
"CORUNDIUM", 
"CORUNDUM", 
"CORUNDUM (AI203)", 
"CORUNDUM (ALUMINUM OXIDE)", 
"CRHYSTALLINE SILICA, QUARTZ", 
"CRISTALLINE SILICA", 
"CRISTALLINE SILICA QUARTZ", 
"CRISTALLINE SILICA, QUARTZ", 
"CRISTALLINE SILLICA: QUARTZ (SIO2)", 
"CRISTOBALITE", 
"CRSTALLINE SILICA (QUARTZ)", 
"CRSTALLINE SILICA, QUARTZ", 
"CRYS4808-60-7TALLINE SIO2", 
"CRYSTAILLINE SILICA: QUARTZ", 
"CRYSTALILINE SILICA: QUARTZ (SIO2)", 
"CRYSTALINE", 
"CRYSTALINE  SILICA, SIO2", 
"CRYSTALINE SAILICA", 
"CRYSTALINE SILCA, SIO2", 
"CRYSTALINE SILICA", 
"CRYSTALINE SILICA ", 
"CRYSTALINE SILICA - QUARTZ", 
"CRYSTALINE SILICA (QUARTZ)", 
"CRYSTALINE SILICA (SIO2)", 
"CRYSTALINE SILICA 100 MESH", 
"CRYSTALINE SILICA 40/70", 
"CRYSTALINE SILICA IN THE FORM OF QUARTZ", 
"CRYSTALINE SILICA QUARTZ", 
"CRYSTALINE SILICA QUARTZ", 
"CRYSTALINE SILICA RESPIRABLE POWDER", 
"CRYSTALINE SILICA SIO2", 
"CRYSTALINE SILICA, CRISTOBALITE", 
"CRYSTALINE SILICA, QUARTZ", 
"CRYSTALINE SILICA, QUARTZ ", 
"CRYSTALINE SILICA, QUARTZ, PHENOL-FORMALDEHYDE NOVOLAK RESIN 9003-35-4, HEXAMEHTYLENETETRAMINE 100-97-0", 
"CRYSTALINE SILICA, QUARTZ, PHENOL-FORMALDEHYDE NOVOLAK RESIN9003-35-4, HEXAMETHYLENETETRAMINE 100-97-0", 
"CRYSTALINE SILICA, RESPIRABLE POWDER", 
"CRYSTALINE SILICA, SIO2", 
"CRYSTALINE SILICA: CRISTOBALITE", 
"CRYSTALINE SILICA: QUARTZ", 
"CRYSTALINE SILICA: QUARTZ (SIO2)", 
"CRYSTALINE SILLICA", 
"CRYSTALINE SILLICA IN FORM OF QUARTZ", 
"CRYSTALIONE SILICA, QUARTZ", 
"CRYSTALLIE SILICA QUARTZ", 
"CRYSTALLIE SILICA, QUARTZ", 
"CRYSTALLIINE SILICA", 
"CRYSTALLIINE SILICA: QUARTZ", 
"CRYSTALLILNE SILICA", 
"CRYSTALLILNE SILICA: QUARTZ (SIO2)", 
"CRYSTALLIME SILICA", 
"CRYSTALLIN SILICA (QUARTZ)", 
"CRYSTALLIN SILICA QUARTZ", 
"CRYSTALLIN SILICA, CRISTOBALITE", 
"CRYSTALLIN SILICA, QUARTZ", 
"CRYSTALLIN SILICA: QUARTZ", 
"CRYSTALLINE", 
"CRYSTALLINE ", 
"CRYSTALLINE CRISTOBALITE", 
"CRYSTALLINE POWDER", 
"CRYSTALLINE POWDWER", 
"CRYSTALLINE QUARTZ", 
"CRYSTALLINE QUARTZ (SILICA)", 
"CRYSTALLINE QUARTZ SILICA", 
"CRYSTALLINE QUARTZ, SILICA", 
"CRYSTALLINE SALT", 
"CRYSTALLINE SI02", 
"CRYSTALLINE SIICA", 
"CRYSTALLINE SIILICA: QUARTZ (SIO2)", 
"CRYSTALLINE SILCA", 
"CRYSTALLINE SILCA (QUARTZ)", 
"CRYSTALLINE SILCA: QUARTZ", 
"CRYSTALLINE SILIC, CRISTOBALITE", 
"CRYSTALLINE SILICA", 
"CRYSTALLINE SILICA ", 
"CRYSTALLINE SILICA  (QUARTZ)", 
"CRYSTALLINE SILICA  QUARTZ", 
"CRYSTALLINE SILICA - QUARTZ", 
"CRYSTALLINE SILICA (<0.02%)", 
"CRYSTALLINE SILICA (CRISOBALITE)", 
"CRYSTALLINE SILICA (CRISTOBALITE)", 
"CRYSTALLINE SILICA (CRISTOBALOITE)", 
"CRYSTALLINE SILICA (CRITOBALITE", 
"CRYSTALLINE SILICA (IMPURITY)", 
"CRYSTALLINE SILICA (IN FORM OF QUARTZ)", 
"CRYSTALLINE SILICA (IN THE FORM OF QUARTZ)", 
"CRYSTALLINE SILICA (QUARTS)", 
"CRYSTALLINE SILICA (QUARTX)", 
"CRYSTALLINE SILICA (QUARTZ", 
"CRYSTALLINE SILICA (QUARTZ SAND SILICON DIOXIDE)", 
"CRYSTALLINE SILICA (QUARTZ SAND, SILICON DIOXIDE)", 
"CRYSTALLINE SILICA (QUARTZ)", 
"CRYSTALLINE SILICA (QUARTZ) ", 
"CRYSTALLINE SILICA (QUARTZ) (SIO2)", 
"CRYSTALLINE SILICA (QUARTZ) SIO2", 
"CRYSTALLINE SILICA (SILICON DIOXIDE)", 
"CRYSTALLINE SILICA (SIO2)", 
"CRYSTALLINE SILICA : QUARTZ (SI02)", 
"CRYSTALLINE SILICA : QUARTZ (SIC02)", 
"CRYSTALLINE SILICA : QUARTZ (SIO2)", 
"CRYSTALLINE SILICA ALKYL", 
"CRYSTALLINE SILICA CRISTOBALITE", 
"CRYSTALLINE SILICA CRISTOBALLITE", 
"CRYSTALLINE SILICA IN FORM OF QUARTZ", 
"CRYSTALLINE SILICA IN THE FORM OF QUARTZ", 
"CRYSTALLINE SILICA IN THE FORM OF QUARTZ ", 
"CRYSTALLINE SILICA IN THE FORM OF QUARZ", 
"CRYSTALLINE SILICA IN THE FORM OR QUARTZ", 
"CRYSTALLINE SILICA IN THE FORMOF QUARTZ", 
"CRYSTALLINE SILICA QUART", 
"CRYSTALLINE SILICA QUARTS", 
"CRYSTALLINE SILICA QUARTZ", 
"CRYSTALLINE SILICA QUARTZ", 
"CRYSTALLINE SILICA QUARTZ", 
"CRYSTALLINE SILICA QUARTZ", 
"CRYSTALLINE SILICA QUARTZ ", 
"CRYSTALLINE SILICA QUARTZ ", 
"CRYSTALLINE SILICA QUARTZ (SI02)", 
"CRYSTALLINE SILICA QUARTZ (SIO2)", 
"CRYSTALLINE SILICA QUARTZ (SIO2)", 
"CRYSTALLINE SILICA QUARTZ (SIO2) ", 
"CRYSTALLINE SILICA QUARTZ(SIO2)", 
"CRYSTALLINE SILICA QUARTZ: SIO2", 
"CRYSTALLINE SILICA QUARZ ", 
"CRYSTALLINE SILICA QUATRTS", 
"CRYSTALLINE SILICA QUATZ", 
"CRYSTALLINE SILICA RESPIRABLE POWDER", 
"CRYSTALLINE SILICA SIO2", 
"CRYSTALLINE SILICA        ",
"CRYSTALLINE SILICA(QUARTZ)", 
"CRYSTALLINE SILICA, (QUARTZ)", 
"CRYSTALLINE SILICA, CRISTOBALITE", 
"CRYSTALLINE SILICA, QARTZ", 
"CRYSTALLINE SILICA, QUART", 
"CRYSTALLINE SILICA, QUARTAZ", 
"CRYSTALLINE SILICA, QUARTS", 
"CRYSTALLINE SILICA, QUARTZ", 
"CRYSTALLINE SILICA, QUARTZ ", 
"CRYSTALLINE SILICA, QUARTZ (SI02)", 
"CRYSTALLINE SILICA, QUARTZ (SIO2)", 
"CRYSTALLINE SILICA, QUARTZ",
"CRYSTALLINE SILICA, QUARTZÂ Â ", 
"CRYSTALLINE SILICA, QUATRTS", 
"CRYSTALLINE SILICA, QUATZ", 
"CRYSTALLINE SILICA, QWUARTZ", 
"CRYSTALLINE SILICA, RESPIRABLE POWDER", 
"CRYSTALLINE SILICA, SIO2", 
"CRYSTALLINE SILICA,QUARTZ", 
"CRYSTALLINE SILICA,QURTZ", 
"CRYSTALLINE SILICA. QUARTZ", 
"CRYSTALLINE SILICA:  CRISTOBALITE", 
"CRYSTALLINE SILICA:  QUARTZ (SIO2)", 
"CRYSTALLINE SILICA:  QUARTZ(SIO2)", 
"CRYSTALLINE SILICA: (SIO2)", 
"CRYSTALLINE SILICA: CHRISTOBALITE", 
"CRYSTALLINE SILICA: CRISTOBALITE", 
"CRYSTALLINE SILICA: CRISTOLBALITE", 
"CRYSTALLINE SILICA: QUART (SIO2)", 
"CRYSTALLINE SILICA: QUARTS", 
"CRYSTALLINE SILICA: QUARTS (SI02)", 
"CRYSTALLINE SILICA: QUARTS (SIO2)", 
"CRYSTALLINE SILICA: QUARTZ", 
"CRYSTALLINE SILICA: QUARTZ ", 
"CRYSTALLINE SILICA: QUARTZ (SI02)", 
"CRYSTALLINE SILICA: QUARTZ (SIO2", 
"CRYSTALLINE SILICA: QUARTZ (SIO2)", 
"CRYSTALLINE SILICA: QUARTZ (SIO2)", 
"CRYSTALLINE SILICA: QUARTZ (SIO2)", 
"CRYSTALLINE SILICA: QUARTZ (SIO2) ", 
"CRYSTALLINE SILICA: QUARTZ (SIO2)",
"CRYSTALLINE SILICA: QUARTZ(SIO2)", 
"CRYSTALLINE SILICA: QUATRZ (SIO2)", 
"CRYSTALLINE SILICA:CRISTOBALITE",
"CRYSTALLINE SILICA:QUARTZ", 
"CRYSTALLINE SILICA:QUARTZ (SIO2)", 
"CRYSTALLINE SILICA; QUARTZ", 
"CRYSTALLINE SILICA;QUARTZ SAND,SILICON DIOXIDE", 
"CRYSTALLINE SILICAM, QUARTZ", 
"CRYSTALLINE SILICAM, QUARTZ ", 
"CRYSTALLINE SILICAQUARTZ", 
"CRYSTALLINE SILICIA QUARTZ", 
"CRYSTALLINE SILICIA, QUARTS", 
"CRYSTALLINE SILICIA, SIO2", 
"CRYSTALLINE SILICIA: QUARTZ (SIO2)", 
"CRYSTALLINE SILICON DIOXIDE", 
"CRYSTALLINE SILLICA", 
"CRYSTALLINE SILLICA ", 
"CRYSTALLINE SILLICA (QUARTZ)", 
"CRYSTALLINE SILLICA (SILICA SAND)", 
"CRYSTALLINE SILLICA IN FORM OF QUARTZ", 
"CRYSTALLINE SILLICA QUARTZ (SIO2)", 
"CRYSTALLINE SILLICA, QUARTZ", 
"CRYSTALLINE SILLICA, QUARTZ (SIO2)", 
"CRYSTALLINE SILLICA: QUARTZ", 
"CRYSTALLINE SILLICA: QUARTZ (SIO2)", 
"CRYSTALLINE SILLICA: QUARTZCRYSTALLINE SILLICA: QUARTZ",
"CRYSTALLINE SIO2", 
"CRYSTALLINE SITICA", 
"CRYSTALLINE SLICA (QUARTZ)", 
"CRYSTALLINE SLICA QUARTZ", 
"CRYSTALLINE SLICA, QUARTZ", 
"CRYSTALLINE SLILCA", 
"CRYSTALLINE SLILICA (QUARTZ)", 
"CRYSTALLINE, SILICA, QUARTZ", 
"CRYSTALLING SILICA, QUARTZ", 
"CRYSTALLLINE SILICA (QUARTZ)", 
"CRYSTALLLINE SILICA QUARTZ", 
"CRYSTALLLINE SILICA QUARTZ (SIO2)", 
"CRYSTALLLINE SILICA, QUARTZ", 
"CRYSTALLLINE SILICA, QUARTZ (SIO2)", 
"CRYSTIALLINE SILICA", 
"CRYSTIALLINE SILICA: CRISTOBALITE", 
"CRYSTIALLINIE SILICA, QUARTZ", 
"CRYSTILLINE SILICA, QUARTZ", 
"CRYSTLLINE SILICA, QUARTZ", 
"CRYTALINE SILICA, CRISTOBALITE", 
"CRYTALLIN SILICA QURATZ", 
"CRYTALLIN SILICA, QURATZ", 
"CRYTALLINE SILICA", 
"CRYTALLINE SILICA (QUARTZ)", 
"CRYTALLINE SILICA, QUARTZ", 
"CRYTALLINE SILICA, SIO2", 
"CS-210PB", 
"CTYSTALLINE SILICA", 
"CTYSTALLINE SILICA", 
"CURED  RESIN", 
"CURED ACRLIC RESIN", 
"CURED ACRLYIC RESIN", 
"CURED ACRYLIC  RESIN", 
"CURED ACRYLIC RESIC", 
"CURED ACRYLIC RESIN", 
"CURED ACRYLIC RESIN ", 
"CURED ACRYLIC RESIN MIXTURE", 
"CURED ACRYLIC RESINQ ", 
"CURED ACRYLIC SAND", 
"CURED ACYLIC RESIN", 
"CURED PHENOLIC RESIN", 
"CURED PHNOLIC RESIN", 
"CURED RESIN", 
"CURED RESIN ", 
"CURED SCRYLIC RESIN", 
"CURED SILICONE RUBBER-POLYDIMETHYLSILOXANE", 
"CURED URETHANE RESIN", 
"CURRED ACRYLIC RESIN", 
"CYRSTALINE SILCA", 
"CYRSTALINE SILICA", 
"CYRSTALLIN SILICA: QUARTZ (SIO2)", 
"CYRSTALLINE SILICA", 
"CYRSTALLINE SILICA QUARTZ", 
"CYRSTALLINE SILICA QUARTZ (SIO2)", 
"CYRSTALLINE SILICA, QUARTZ", 
"CYRSTALLINE SILICA, QUARTZ (SIO2)", 
"CYRSTALLINE SILICA: QUARTZ (SIO2)", 
"CYRTALLINE SILICA QUARTZ", 
"CYSTALLINE SALICA, SIO2", 
"CYSTALLINE SILICA", 
"CYSTALLINE SILICA (QUARTZ)", 
"CYSTALLINE SILICA QUARTZ", 
"CYSTALLINE SILICA, (QUARTZ)", 
"CYSTALLINE SILICA, QUARTZ", 
"CYSTALLINE SILICA: QUARTZ", 
"CYSTALLINE SILICA; QUARTZ", 
"FRAC SAND", 
"FUMED SILICA", 
"GLASS FIBERS", 
"GLASS OXIDE", 
"GLASS OXIDE; GLASS", 
"GLASS, OXIDE", 
"GLASSY CALCIUM MAGNESIUM PHOSPHATE", 
"GLASSY CALSIUM MAGNESIUM PHOSPHATE", 
"ILLITE", 
"ILMENITE", 
"IRON OXIDE", 
"IRON OXIDE ", 
"IRON OXIDE (COLORANT)", 
"IRON OXIDE (FUME)", 
"LIMESTONE", 
"MICA", 
"MODIFIED BENTONITE", 
"MODIFIED BENTONITE (ORGANOPHILLIC CLAY)", 
"MODIFIED BENTONITE CLAY", 
"MUILLITE", 
"MULITE", 
"MULITIE", 
"MULLITE", 
"MULLITE ", 
"MULLITE (ALUMINASILICATE)", 
"MULLITE (ALUMINUM SILICATE)", 
"MULLITE (AMMONIUM SILICATE)", 
"NON- CRYSTALLINE SILICA (IMPURITY)", 
"NON-CRYSTALLINE SILICA (IMPURITY)", 
"NON-CRYSTALLINE SUBSTANCE", 
"OSYALKYLATED RESINS", 
"OTHER INORGANIC MAGNESIUM COMPOUND", 
"OTHER INORGANIC MAGNESIUM CPD", 
"OXIDE POLYMER ", 
"OXIDES OF SILICA, IRON, ALUMINUM", 
"P/F NOVOLAK RESIN", 
"P/F RESIN", 
"POLACTIDE  RESIN", 
"POLACTIDE RESIN", 
"POLIYLACTIDE RESIN", 
"POLLACTIDE RESIN", 
"POLLYLACTIDE RESIN", 
"POLULACTIDE RESIN", 
"POLYACETIDE RESIN", 
"POLYACITE RESIN", 
"POLYACITIDE RESIN", 
"POLYACITIDE RESIN ", 
"POLYACTDE RESIN", 
"POLYACTIDE", 
"POLYACTIDE RESIGN", 
"POLYACTIDE RESIN", 
"POLYACTIDE RESIN ", 
"POLYACTIDE RESUN", 
"POLYACTITE RESIN", 
"POLYACTITIDE RESIN", 
"POLYALACTIDE RESIN", 
"POLYCITE RESIN", 
"POLYERETHANE RESIN", 
"POLYEURETHANE RESIN", 
"POLYLACIDITE RESIN", 
"POLYLACITE RESIN", 
"POLYLACITIDE RESIN", 
"POLYLACTACTIDE RESIN", 
"POLYLACTIC RESIN", 
"POLYLACTICDE RESIN", 
"POLYLACTICIDE RESIN", 
"POLYLACTID RESIN", 
"POLYLACTIDE", 
"POLYLACTIDE  RESIN", 
"POLYLACTIDE RESEN", 
"POLYLACTIDE RESID", 
"POLYLACTIDE RESIGN", 
"POLYLACTIDE RESIN", 
"POLYLACTIDE RESIN ", 
"POLYLACTIDEE RESIN", 
"POLYLACTITE RESIN", 
"POLYLACTIVE RESIN", 
"POLYLASTIDE RESIN", 
"PREMIUM WHITE 40/70 PROPPANT", 
"PROPRIETARY SILICA", 
"PROPRIETARY SILICA ORGANIC POLYMER", 
"QUARTS (SIO2)", 
"QUARTZ", 
"QUARTZ  (CRYSTALLINE SILICA)", 
"QUARTZ - SILICA", 
"QUARTZ (CRYSTALLINE SILICA", 
"QUARTZ (CRYSTALLINE SILICA)", 
"QUARTZ (CRYSTILLANE SILICA)", 
"QUARTZ (SI02)", 
"QUARTZ (SI02) (CRYSTALLINE SILICA)", 
"QUARTZ (SILICA)", 
"QUARTZ (SIO2)", 
"QUARTZ (SIO2) ", 
"QUARTZ (SIO2)  (CRYSTALLINE SILICA)", 
"QUARTZ (SIO2) (CRYSTALLINE SILICA", 
"QUARTZ (SIO2) (CRYSTALLINE SILICA)", 
"QUARTZ (SIO2) (CRYSTALLINE SILICA) ", 
"QUARTZ (SIO2) (CRYSTASLLINE SILICA)", 
"QUARTZ (SIO2) (CYRSTALLINE SILICA)", 
"QUARTZ (SIO2) CRYSTALLINE SILLICA", 
"QUARTZ (SIO2)(CRYSTALLINE SILICA)", 
"QUARTZ CRYSTALLINE SILICA", 
"QUARTZ CRYSTALLINE SILICA ", 
"QUARTZ -SILICA", 
"QUARTZ(SIO2)", 
"QUARTZ, CRYSTALLINE SILICA", 
"QUARTZ, CRYSTALLINE SILICA ", 
"QUARTZ, CRYSTALLINE SILLICA", 
"QUARTZ, SIO2", 
"RESIN", 
"RESIN A", 
"RESIN COATED CRYSTALLINE SILICA ", 
"RESIN COATED CRYSTALLINE SILICA (QUARTZ)", 
"RESIN COATED FRACTURING PROPPANT", 
"RESIN COATED PROPPANT", 
"RESIN COATED SILICA FRACTURING PROPPANT", 
"RESIN COMPOUND", 
"RESPIRABLE CRYSTALLING SILICA (QUARTS)", 
"RYSTALLINE SILICA", 
"RYSTALLINE SILICA, QUARTZ", 
"SAND", 
"SAND - CRYSTALINE SILICA", 
"SAND - CRYSTALLINE SILICA", 
"SAND (CRYSTALLINE SILICA)", 
"SAND 20/40 OTTOWA WHITE", 
"SAND-PREMIUM WHITE", 
"SI", 
"SI-2 BLEND", 
"SICLA QUARTZ", 
"SIICIA QUARTZ", 
"SIICIA, QUARTZ", 
"SIILICA AMORPHOUS PRECIPITATED AND GEL", 
"SIILICA, AMORPHOUS PRECIPITATED AND GEL", 
"SILCA", 
"SILCA, CRYSTALLINE-QUARTZ", 
"SILCON DIOXIDE", 
"SILCON DIOXIDE (SILICA SAND)", 
"SILCON DIOXIDE AMORPHOUS AS GLASS", 
"SILCON DIOXIDE, (AMORPHOUS AS GLASS)", 
"SILCON DIOXIDE, AMORPHOUS AS GLASS", 
"SILICA", 
"SILICA", 
"SILICA ", 
"SILICA - CRYSTALLINE QUARTZ", 
"SILICA - CRYSTALLINE, QUARTZ", 
"SILICA (CRYSTALINE)", 
"SILICA (QUARTZ) SIO2", 
"SILICA , CRYSTALLINE - QUARTZ", 
"SILICA , CRYSTALLINE QUARTZ", 
"SILICA , CRYSTALLINE-QUARTZ", 
"SILICA AMORPHOUS", 
"SILICA AMORPHOUS - FUMED", 
"SILICA AMORPHOUS FUMED CRYSTAL FREE", 
"SILICA AMORPHOUS FUMED CRYSTALLINE - FREE", 
"SILICA AMORPHOUS FUSED", 
"SILICA AMORPHOUS PRECIPIDATED AND GEL", 
"SILICA AMORPHOUS PRECIPITATE", 
"SILICA AMORPHOUS PRECIPITATED", 
"SILICA AMORPHOUS PRECIPITATED AND GEL", 
"SILICA AMORPHOUS-FUMED", 
"SILICA CHRYSTALINE CRISTOBALITE", 
"SILICA CRISTOBALITE", 
"SILICA CRYATALLINE QUARTZ", 
"SILICA CRYSALLINE QUARTZ", 
"SILICA CRYSLLINE QUARTZ", 
"SILICA CRYSTALINE QUARTZ", 
"SILICA CRYSTALINE-CRISTOBALITE", 
"SILICA CRYSTALINE-QUARTZ", 
"SILICA CRYSTALLINE", 
"SILICA CRYSTALLINE - CRISTABOLITE", 
"SILICA CRYSTALLINE - QUARTZ", 
"SILICA CRYSTALLINE (QUARTZ)", 
"SILICA CRYSTALLINE CRISTOBALITE", 
"SILICA CRYSTALLINE -CRISTOBALITE", 
"SILICA CRYSTALLINE QUARTZ", 
"SILICA CRYSTALLINE QUARTZ", 
"SILICA CRYSTALLINE -QUARTZ", 
"SILICA CRYSTALLINE- QUARTZ", 
"SILICA CRYSTALLINE-CRISTBALITE", 
"SILICA CRYSTALLINE-CRISTOBALIT", 
"SILICA CRYSTALLINE-CRISTOBALITE", 
"SILICA CRYSTALLINE-CRISTOBALLITE", 
"SILICA CRYSTALLINE-CRISTOBALT", 
"SILICA CRYSTALLINE-CRITOBALITE", 
"SILICA CRYSTALLINE-QUARTZ", 
"SILICA CRYSTALLLINE-CRISTOBALITE", 
"SILICA CRYSTANLLLINE", 
"SILICA CYSTALLINE - CRISTOBALITE", 
"SILICA CYSTALLINE - QUARTZ", 
"SILICA CYSTALLINE CRISTOBALITE", 
"SILICA CYSTALLINE-QUARTZ", 
"SILICA DIOXIDE", 
"SILICA DISPERSION", 
"SILICA ENHANCED FLAKES WITH ACIDIC RESIDUAL", 
"SILICA GEL", 
"SILICA GEL, PPTD., CRYST.-FREE", 
"SILICA IN FORM OF QUARTZ", 
"SILICA ORGANIC POLYMER", 
"SILICA QUARTS", 
"SILICA QUARTZ", 
"SILICA QUARTZ", 
"SILICA QUARTZ SI02", 
"SILICA QUARTZ SIO2", 
"SILICA QUARTZ SIO2", 
"SILICA QUARTZ SIO2", 
"SILICA QUARTZ, SIO2", 
"SILICA QUARTZ, SIO2 ", 
"SILICA QUARTZ, SIO2/PHENOLIC RESIN/HEXAMETHYLENETETRAMINE", 
"SILICA QUARTZ, SIO5", 
"SILICA QUARTZ/ SIO2", 
"SILICA SAND", 
"SILICA SAND ", 
"SILICA SAND (QUARTZ)", 
"SILICA SAND(QUARTZ)", 
"SILICA SANDS (QUARTZ)", 
"SILICA SUBSTATE", 
"SILICA SUBSTITUTE WITH BONDED COATINGS", 
"SILICA SUBSTRAIT", 
"SILICA SUBSTRAT", 
"SILICA SUBSTRATE", 
"SILICA SUBSTRATTE", 
"SILICA SUBTRATE", 
"SILICA, AMORPHOUS", 
"SILICA, AMORPHOUS - FUMED", 
"SILICA, AMORPHOUS - FUMED ", 
"SILICA, AMORPHOUS- FUMED", 
"SILICA, AMORPHOUS FUSED", 
"SILICA, AMORPHOUS PRECIPIDATED AND GEL", 
"SILICA, AMORPHOUS PRECIPITATE", 
"SILICA, AMORPHOUS PRECIPITATED", 
"SILICA, AMORPHOUS PRECIPITATED AND GEL", 
"SILICA, AMORPHOUS, FUMED, CRYSTAL FREE", 
"SILICA, AMORPHOUS, FUMED, CRYSTALLINE - FREE", 
"SILICA, AMORPHOUS-FUMED", 
"SILICA, AMPORHOUS - FUMED", 
"SILICA, CRISTOBALITE", 
"SILICA, CRYATALINE-QUARTZ", 
"SILICA, CRYATALLINE-QUARTZ", 
"SILICA, CRYATALLINE-QUARTZ ", 
"SILICA, CRYSTALINE QUARTZ", 
"SILICA, CRYSTALINE, QUARTZ", 
"SILICA, CRYSTALINE-QUARTZ", 
"SILICA, CRYSTALLINE", 
"SILICA, CRYSTALLINE - QUARTZ", 
"SILICA, CRYSTALLINE - QUARTZ ", 
"SILICA, CRYSTALLINE (CRISTOBALITE)", 
"SILICA, CRYSTALLINE (QUARTZ)", 
"SILICA, CRYSTALLINE- QUARTSQUARTZQUARTZ",
"SILICA, CRYSTALLINE QUARTZ", 
"SILICA, CRYSTALLINE- QUARTZ", 
"SILICA, CRYSTALLINE, QUARTZ", 
"SILICA, CRYSTALLINE-/QUARTZ", 
"SILICA, CRYSTALLINE-CRISTOBALITE", 
"SILICA, CRYSTALLINE-QUARTZ", 
"SILICA, CRYTALLINE QUARTZ", 
"SILICA, CYSTALLINE - QUARTZ", 
"SILICA, CYSTALLINE-QUARTZ", 
"SILICA, QUART, SIO2", 
"SILICA, QUARTZ", 
"SILICA, QUARTZ SIO2", 
"SILICA, QUARTZ, S102", 
"SILICA, QUARTZ, SIO2", 
"SILICA, QUARTZ,SIO2", 
"SILICA,CRYSTALLINE QUARTZ", 
"SILICA,QUARTZ", 
"SILICA/CRYSTALLINE QUARTZ", 
"SILICA/QUARTZ/SIO2", 
"SILICA-CRYSTALLINE QUARTZ", 
"SILICA-CRYSTALLINE, QUARTZ", 
"SILICA-CRYSTALLINE-CRISTOBALITE", 
"SILICAN CRYSTALLINE-CRISTOBALITE", 
"SILICATE MINERALS - TS", 
"SILICIA CRYSTALINE-CRISTOBALITE", 
"SILICIA CRYSTALLINE QUARTZ", 
"SILICIA, CRYSTALINE QUARTZ", 
"SILICIA, CRYSTALLINE QUARTZ", 
"SILICON", 
"SILICON  DIOXIDE", 
"SILICON DIOXIDE", 
"SILICON DIOXIDE ", 
"SILICON DIOXIDE (AMORPHOUS AS GLASS)", 
"SILICON DIOXIDE (AMORPHOUS AS GLASS)", 
"SILICON DIOXIDE (AMORPHOUS)", 
"SILICON DIOXIDE (CRYSTALLINE)", 
"SILICON DIOXIDE (QUARTZ)", 
"SILICON DIOXIDE (SILICA SAND)", 
"SILICON DIOXIDE (SILICIA SAND)", 
"SILICON DIOXIDE (SIO2)", 
"SILICON DIOXIDE AMORPHOUS AS GLASS", 
"SILICON DIOXIDE CHEMICALLY PREPARED", 
"SILICON DIOXIDE IN THE FORM OF CRYSTALLINE SILICA", 
"SILICON DIOXIDE IN THE FORM OF CRYSTALLINE SILICA (QUARTZ)", 
"SILICON DIOXIDE(AMORPHOUS AS GLASS)", 
"SILICON DIOXIDE(AMORPHOUS AS GLASS)", 
"SILICON DIOXIDE, (AMORPHOUS AS GLASS)", 
"SILICON DIOXIDE, AMORPHOUS - FUMED", 
"SILICON DIOXIDE, AMORPHOUS AS GLASS", 
"SILICON DIOXIDE, CHEMICALLY PREPARED", 
"SILICON DIOXIDE, QUARTZ", 
"SILICON DIOXIDE,(AMORPHOUS AS GLASS)", 
"SILICON DIOXIDE. (AMPORPHOUS AS GLASS)", 
"SILICON DIOXIDE...", 
"SILICON DIOXODE", 
"SILICON DIXIDE, (AMORPHOUS AS GLASS)", 
"SILICONE DIOXIDE", 
"SILICONE DIOXIDE ", 
"SILICONE DIOXIDE, (AMORPHOUS AS GLASS)", 
"SILLICA AMORPHOUS PRECIPITATED AND GEL", 
"SILLICA CRYSTALLINE QUARTZ", 
"SILLICA, AMORPHOUS PRECIPITATED AND GEL", 
"SILLICA, CRYSTALLINE QUARTZ", 
"SILLICA, QUARTZ, SIO2", 
"SILLICON DIOXIDE", 
"SINTERED BAUXITE", 
"SIO2", 
"SYNTHETIC AMORPHOUS METAL SILICATE - TS", 
"SYNTHETIC AMORPHOUS SILICA (CRYSTALLINE SILICA-FREE)", 
"UEXITE", 
"ULEXIE", 
"ULEXITE", 
"ULEXITE", 
"ULEXITE ", 
"ULEXITE - TS", 
"ULEXITE (BORATE SALT)", 
"ULEXITE CALCINED", 
"ULEXITE MINERAL", 
"ULEXITE MINERAL ", 
"ULEXITE POWDER", 
"ULEXITE POWER", 
"ULEXITEULEXITEULEXITE", 
"ULEXITE, CALCINED", 
"ULIXITE MINERAL", 
"ULLEXITE"]

In [ ]:
FFPropvol = FFPropvol[FFPropvol["IngredientName"].isin(Proppantlist)]
FFPropvol = FFPropvol[["APINumber", "PercentHFJob"]]
FFPropvol['ProppantPerc'] = FFPropvol["PercentHFJob"].groupby(FFPropvol['APINumber']).transform('sum')
FFPropvol = FFPropvol.drop_duplicates(subset="APINumber", keep='first', inplace=False)
FFPropvol = FFPropvol.reset_index(drop=True)

In [ ]:
#Put together your water volume/general info and propant dataframes
FFCalcFrame = pd.merge(FFwatervol, FFPropvol, on='APINumber')

#Clean up Kern County format
FFCalcFrame['CountyName'] = FFCalcFrame['CountyName'].replace(' - 30','', regex=True)

#Capitalize all Operators
FFCalcFrame['OperatorName'] = FFCalcFrame['OperatorName'].str.upper()

#Add a year column to break the data out by a timeframe
FFCalcFrame['FracYear'] = FFCalcFrame['JobEndDate'].dt.year.astype(int)

#Drop PercentHFJob column
FFCalcFrame = FFCalcFrame.drop(['PercentHFJob'], axis=1)

In [ ]:
#Get a gist of how much Proppant is being used by percent in your "cleaner" dataframe.
#Some numbers are still running high, but we will clean that up in a bit.

plt.hist(FFCalcFrame['ProppantPerc'], bins = 10, range = (0,100));

In [ ]:
FFCalcFrame["FracFluid_Weight_Pct"] = 100-FFCalcFrame["ProppantPerc"]
FFCalcFrame["TBV_Water_Weight"] = FFCalcFrame["TotalBaseWaterVolume"]*8.34
FFCalcFrame["Total_Frac_Weight_(lbs.)"] = FFCalcFrame["TBV_Water_Weight"]/(FFCalcFrame["FracFluid_Weight_Pct"]/100.0)
FFCalcFrame["Prop_Weight_(lbs.)"] = FFCalcFrame["Total_Frac_Weight_(lbs.)"]*(FFCalcFrame["ProppantPerc"]/100.0)
FFCalcFrame["FracFluidWeight_(lbs.)"] = FFCalcFrame["TBV_Water_Weight"]*(FFCalcFrame["FracFluid_Weight_Pct"]/100.0)
FFCalcFrame["Frac_Fluid_Barrels_(bbls)"] = FFCalcFrame["TotalBaseWaterVolume"]/42.0
FFCalcFrame

In [ ]:
FFCalcFrame.describe()

In [ ]:
import seaborn as sns; sns.set(style="ticks", color_codes=True)

FFCalcFrameNum = FFCalcFrame[['TotalBaseWaterVolume','TBV_Water_Weight', 'Prop_Weight_(lbs.)', 'FracFluidWeight_(lbs.)', 'Total_Frac_Weight_(lbs.)']]
FFpairplt = sns.pairplot(FFCalcFrameNum)
FFpairplt

-Looks like there may be some crazy numbers in the data set, so let's cut them.

-We show below one way to statistically set where outliers start occuring by using the IQR and subtracting and adding 1.5 times the IQR from the respective quartiles

-We know there are a lot of high prop/fluid wells out there (some with 50 million lbs of sand), and in a data set running from 2009 to present, there will be some definite huge swings on values.

-It would be easier to make some calls on this if you had lateral lengths, but you can fix this up if you narrow your 
project down to a state or 2 and get that lateral data for the calculation. 

-So, from 0 to 23,238,640 gallons on total base water volume is the range.

In [ ]:
#Example of how to use IQR in Python

#Get the interquartile range on Total Base Water Volume
q75, q25 = np.percentile(FFCalcFrame['TotalBaseWaterVolume'], [75 ,25])
TBV_iqr = q75 - q25

#Use the iqr to set bounds on where to call outliers
TBV_LB = q25 - (1.5*TBV_iqr)
TBV_UB = q75 + (1.5*TBV_iqr)
print("IQR:",TBV_iqr, "Lower Bound:", TBV_LB, "Upper Bound:", TBV_UB)


In [ ]:
FFCalcFrame2 = FFCalcFrame[FFCalcFrame['TotalBaseWaterVolume'] <= TBV_UB]
FFCalcFrame2.describe()

In [ ]:
FFCalcFrameNum2 = FFCalcFrame2[['TotalBaseWaterVolume','TBV_Water_Weight', 'Prop_Weight_(lbs.)', 'FracFluidWeight_(lbs.)', 'Total_Frac_Weight_(lbs.)']]
FFpairplt = sns.pairplot(FFCalcFrameNum2)
FFpairplt

Much better, but that ProppantPerc, and therefore proppant weight, is still insanely high.  Do the same thing as before using the IQR.

In [ ]:
PWPq75, PWPq25 = np.percentile(FFCalcFrame['Prop_Weight_(lbs.)'], [75 ,25])
PWP_iqr = PWPq75 - PWPq25

PWP_LB = q25 - (1.5*PWP_iqr)
PWP_UB = q75 + (1.5*PWP_iqr)
print("IQR:",PWP_iqr, "Lower Bound:", PWP_LB, "Upper Bound:", PWP_UB)

In [ ]:
FFCalcFrame3 = FFCalcFrame2[FFCalcFrame2['Prop_Weight_(lbs.)'] <= PWP_UB]
FFCalcFrame3.describe()

In [ ]:
FFCalcFrameNum3 = FFCalcFrame3[['TotalBaseWaterVolume','TBV_Water_Weight', 'Prop_Weight_(lbs.)', 'FracFluidWeight_(lbs.)', 'Total_Frac_Weight_(lbs.)']]
FFpairplt = sns.pairplot(FFCalcFrameNum3)
FFpairplt

In [ ]:
FFCalcFrame3